# Pets example 

## Event based approach

Let's simulate the easy life of a pet in the morning. First we define some data structures for pets:

In [1]:
using DiscreteEvents, Random, Printf

abstract type PetState end
struct Sleeping  <: PetState end
struct Scuffing  <: PetState end
struct Running   <: PetState end

abstract type PetEvent end
struct GetWeary  <: PetEvent end
struct GetHungry <: PetEvent end
struct Scuff     <: PetEvent end
struct LeapUp    <: PetEvent end
struct Sleep     <: PetEvent end

mutable struct Pet
    clk::Clock
    name::String
    state::PetState
    speak::String
end

Then we need an API for Pet since we do not want to access its state directly.

In [17]:
state(p) = p.state
speak(p, n) = @printf("%5.2f %s: %s\n", tau(p.clk), p.name, p.speak^n)

function setstate!(p::Pet, q::PetState)
    p.state = q
    @printf("%5.2f %s: %s\n", tau(p.clk), p.name, repr(p.state))
end

setstate! (generic function with 1 method)

We describe the behaviour of the pet with some transition functions:

In [14]:
function doit!(p::Pet, ::Sleeping, ::LeapUp)   # leap up after sleeping
    setstate!(p, Running())
    event!(p.clk, fun(doit!, p, fun(state, p), GetHungry()), after, 5*rand())
end

function doit!(p::Pet, ::Scuffing, ::LeapUp)   # leap up while scuffing
    setstate!(p, Running())
    event!(p.clk, fun(doit!, p, fun(state, p), GetWeary()), after, 2*rand())
end

function doit!(p::Pet, ::Running, ::GetHungry) # get hungry while running
    speak(p, 5)
    event!(p.clk, fun(doit!, p, fun(state, p), Scuff()), after, rand())
end 

function doit!(p::Pet, ::Running, ::GetWeary)  # get weary while running
    speak(p, 2)
    event!(p.clk, fun(doit!, p, fun(state, p), Sleep()), after, 2*rand())
end 

function doit!(p::Pet, ::Running, ::Scuff)     # scuff after running
    setstate!(p, Scuffing())
    event!(p.clk, fun(doit!, p, fun(state, p), LeapUp()), after, 2*rand())
end

function doit!(p::Pet, ::Running, ::Sleep)     # sleep after running
    setstate!(p, Sleeping())
    event!(p.clk, fun(doit!, p, fun(state, p), LeapUp()), after, 10*rand())
end

doit! (generic function with 6 methods)

Then we setup a simulation clock and a pet, schedule the first event and run it:

In [18]:
clk = Clock()
Random.seed!(123)
snoopy = Pet(clk, "Snoopy", Sleeping(), "huff")
event!(clk, fun(doit!, snoopy, snoopy.state, LeapUp()), after, 5)
run!(clk, 25)

 5.00 Snoopy: Running()
 8.84 Snoopy: huffhuffhuffhuffhuff
 9.78 Snoopy: Scuffing()
11.13 Snoopy: Running()
11.92 Snoopy: huffhuff
12.55 Snoopy: Sleeping()
19.17 Snoopy: Running()
22.10 Snoopy: huffhuffhuffhuffhuff
22.16 Snoopy: Scuffing()
22.69 Snoopy: Running()
22.91 Snoopy: huffhuff
23.24 Snoopy: Sleeping()


"run! finished with 12 clock events, 0 sample steps, simulation time: 25.0"

## Processes and implicit events

`DiscreteEvents` provides us also with another approach: process-based simulation. In this case we implement the pet behaviour in a single function. For such a simple example this comes out simpler and more convenient:

In [27]:
function pet(clk::Clock, p::Pet)
    setstate!(p, Running());  delay!(clk,  5*rand())
    speak(p, 5);              delay!(clk,    rand())  # get hungry
    setstate!(p, Scuffing()); delay!(clk,  2*rand())
    setstate!(p, Running());  delay!(clk,  2*rand())
    speak(p, 2);              delay!(clk,  2*rand())  # get weary
    setstate!(p, Sleeping()); delay!(clk, 10*rand())
end

pet (generic function with 1 method)

This describes one pet cycle. After each status change the pet function `delay!`s (gets suspended) for a given timeout on the clock. 

We have to reimplement our `speak` and `setstate!` functions since now we print from an asynchronous process. With `now!` we let the clock do the printing: 

In [29]:
speak(p, n) = now!(p.clk, fun(println, @sprintf("%5.2f %s: %s", tau(p.clk), p.name, p.speak^n)))

function setstate!(p::Pet, q::PetState)
    p.state = q
    now!(p.clk, fun(println, @sprintf("%5.2f %s: %s", tau(p.clk), p.name, repr(p.state))))
end

setstate! (generic function with 1 method)

In order to make this work we have to register the pet function to the clock.

In [28]:
resetClock!(clk, t0=5)              # reset the clock, we start at 5
Random.seed!(123)                   # reseed the random generator
setstate!(snoopy, Sleeping())       # set snoopy sleeping
process!(clk, Prc(1, pet, snoopy));
run!(clk, 20)

 5.00 Snoopy: Sleeping()
 5.00 Snoopy: Running()
 8.84 Snoopy: huffhuffhuffhuffhuff
 9.78 Snoopy: Scuffing()
11.13 Snoopy: Running()
11.92 Snoopy: huffhuff
12.55 Snoopy: Sleeping()
19.17 Snoopy: Running()
22.10 Snoopy: huffhuffhuffhuffhuff
22.16 Snoopy: Scuffing()
22.69 Snoopy: Running()
22.91 Snoopy: huffhuff
23.24 Snoopy: Sleeping()


"run! finished with 24 clock events, 0 sample steps, simulation time: 25.0"

We got the same output – with more events for the `delay!` and `now!` calls).

## Evaluation

There is no point in doing simulations with such simple sequential examples, but if we do the same with more pets operating in parallel, things get messy very quickly and there is no way to code it sequentially. For every different random number seed we get different sequences of events. If we want to simulate that, we need parallel state machines, processes, actors … and their coordination on a time line. 